In [1]:
import pandas as pd
import numpy as np
import pymysql
import time
import datetime
import joblib
import re
import difflib

In [138]:
joblib.dump(set(llit),'llit.pkl')

['llit.pkl']

In [135]:
joblib.dump(calss_category_dict,'calss_category_dict.pkl')

['calss_category_dict.pkl']

In [2]:
def gp_spider():
    conn = pymysql.connect(
        host="
        user="",
        password="",
        db="master_risk",
        charset="utf8")
    sql = f"""
                SELECT
                * from t_gp_spider where status != 404
            """

    data = pd.read_sql(sql, conn)
    conn.close()
    return data

In [3]:
class Clean_data:
    def __init__(self,data=None,x=None):
        self.x=x
    def install (self,x):
        try:
            if not x:
                return np.NaN
            return int(x.replace(",", "").replace("+", ""))
        except:
            return np.NaN
    def clean_data(self):
        data=gp_spider()
        #data["package_name"]=data.download_andriod.apply(lambda x : x.split("=")[1])
        data["package_name"]=data.download_andriod.apply(lambda x : x[(x.find("=")+1):])
        data["review_total"]=data["review_total"].apply(lambda x :  np.NaN if not x else int(x.replace(",","")))
        data["install_total"] = data["install_total"].apply(self.install)
        data["score"]=data["score"].astype("float")
        return data

In [4]:
%%time
gp_data=Clean_data().clean_data()

CPU times: user 6.25 s, sys: 506 ms, total: 6.76 s
Wall time: 17.9 s


In [30]:
gp_data['package_name']=gp_data['andriod_name']

In [51]:
gp_data

,id,app_name,is_used,provider,category,review_total,install_total,app_logo_url,app_version,version,...,status,create_date,update_date,remark,merchant_code,sort,andriod_name,package_name,class_category,describes_data
0,1,TikTok,1,TikTok Pte. Ltd.,视频播放和编辑,11032399.0,100000000.0,https://play-lh.googleusercontent.com/2kdv4gGW...,None,V1,...,200,2021-07-30 13:15:49,2021-07-31 11:07:30,None,api_loansuper,13,com.ss.android.ugc.trill,com.ss.android.ugc.trill,Music_audio,0.019667
1,2,Rupiah Cepat- Pinjaman Uang Tunai Kredit Dana ...,1,PT.KUFI,财务,934235.0,10000000.0,https://play-lh.googleusercontent.com/2uZ3WWd9...,2.4.5,V1,...,200,2021-07-30 13:15:49,NaT,None,api_loansuper,11,com.loan.cash.credit.easy.kilat.cepat.pinjam.u...,com.loan.cash.credit.easy.kilat.cepat.pinjam.u...,Finance,0.714465
2,3,MyTelkomsel - Beli Pulsa/Paket & dapat kuota 7...,1,Telkomsel,工具,5184414.0,50000000.0,https://play-lh.googleusercontent.com/DBzJQ2z8...,6.1.0,V1,...,200,2021-07-30 13:15:49,NaT,None,api_loansuper,22,com.telkomsel.telkomselcm,com.telkomsel.telkomselcm,Office,0.664570
3,4,Kredivo - Cicilan Tanpa Kartu dan Pinjaman Tunai,1,PT FinAccel Finance Indonesia,财务,1474364.0,NaN,https://play-lh.googleusercontent.com/K2PGdTgv...,None,V1,...,200,2021-07-30 13:15:49,NaT,None,api_loansuper,18,com.finaccel.android,com.finaccel.android,Finance,0.767469
4,5,UATAS: Pinjaman Uang Tunai Dana Online,1,PT. Plus Ultra Abadi,财务,85459.0,500000.0,https://play-lh.googleusercontent.com/2XOO_Ukd...,None,V1,...,200,2021-07-30 13:15:50,2021-07-31 11:11:17,None,api_loansuper,30,com.uatas.android,com.uatas.android,Finance,0.681293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121059,212749,Monster Truck Crash Stunts: Demolition Derby 2021,1,Vividd Games,天气,213.0,100000.0,https://play-lh.googleusercontent.com/6wgfEbSs...,1.8,V1,...,200,2021-08-04 12:32:39,NaT,None,api_loansuper_x5,52947,com.vividdgames.monster.truck.jam.driving.madt...,com.vividdgames.monster.truck.jam.driving.madt...,Travel_vehicles,0.604229
121060,212750,迪诺侏罗纪工艺:进化&立方体探索3D,1,"Tiny Dragon Adventure Games: Craft, Sport & RPG",模拟,23296.0,5000000.0,https://play-lh.googleusercontent.com/axsl9o-V...,因设备而异,V1,...,200,2021-08-04 12:32:40,NaT,None,api_loansuper_x5,52950,com.tinydragonadventuregames.mine.craft.pocket...,com.tinydragonadventuregames.mine.craft.pocket...,Game,0.045547
121061,212751,男孩世界：创造思维和探索,1,Crafting And Building Games For Girls Adventure,冒险,30641.0,10000000.0,https://play-lh.googleusercontent.com/3QQ2jPWT...,None,V1,...,200,2021-08-04 12:32:40,NaT,None,api_loansuper_x5,52959,com.buildingcraftinggames.fun.kid.games.boys.b...,com.buildingcraftinggames.fun.kid.games.boys.b...,Game,0.042068
121062,212752,Free Diamonds Spin & Elite Pass Calc For Free ...,1,"DAimond and Vbuks , Free UC and pass",图书与工具书,1126.0,100000.0,https://play-lh.googleusercontent.com/ZKHeMLKR...,None,V1,...,200,2021-08-04 12:32:40,NaT,None,api_loansuper_x5,52961,freediamondsforff.free_fire_max.game.freefireg...,freediamondsforff.free_fire_max.game.freefireg...,Education,0.587430


In [5]:
class_category = { "通讯": "Communications",
                  "社交": "Communications",
                  "购物": "Lifestyle",
                  "效率":"Office",
                  "视频播放和编辑": "Music_audio",
                  "摄影": "Music_audio",
                  "理财":"Finance",
                  "工具": "Office", 
                  "娱乐 ": "Entertainment", 
                  "音乐与音频": "Music_audio",
                  "地图和导航": "Travel_vehicles",
                  "新闻杂志": "Lifestyle", 
                  "动作": "Game",
                  '办公':'Office',
                  "旅游与本地出行": "Travel_vehicles",
                  "个性定制": "Lifestyle",
                  "医疗": "Medical",
                  "图书与工具书":"Education",
                  "桌面和棋类":"Game",
                  "生活时尚": "Lifestyle",
                  "健康与健身": "Lifestyle",
                  "休闲": "Lifestyle",
                  "教育": "Education",
                  "模拟": "Game", 
                  "天气": "Travel_vehicles",
                  "益智": "Game",
                  "街机": "Game",
                  "体育": "Game",
                  "餐饮美食": "Lifestyle",
                  "育儿": "Education",
                  "冒险": "Game",
                  "卡牌": "Game",
                  "文字": "Game",
                  "知识问答": "Game",
                  "音乐": "Music_audio",
                   "策略": "Game",
                  "赌场": "Game",
                  "艺术和设计": "Lifestyle",
                  "家居装修": "Lifestyle",
                  "漫画": "Game",
                  "竞速": "Game",
                  "软件库与演示": "Lifestyle",
                  "角色扮演": "Game",
                  "美容时尚": "Lifestyle",
                  "社交约会": "Communications",
                  "活动": "Lifestyle",
                  "财务": "Finance",
                  "公司": "Office",
                  "商务办公": "Office",
                  "个性化": "Lifestyle",
                  "效率应用": "Office"}

In [31]:
gp_data["class_category"] = gp_data["category"].map(class_category).fillna("others")

gp_class_category = gp_data["class_category"].tolist()

review_app={}

review_app['review_unpopular_app']= gp_data[
    gp_data.review_total < gp_data.review_total.quantile(0.2)]["package_name"].tolist()
review_app['review_unpopular_second_app'] = gp_data[(gp_data.review_total >= gp_data.review_total.quantile(0.2)) & (
        gp_data.review_total < gp_data.review_total.quantile(0.4))]["package_name"].tolist()
review_app['review_middle1_app'] = gp_data[(gp_data.review_total >= gp_data.review_total.quantile(0.4)) & (
        gp_data.review_total < gp_data.review_total.quantile(0.6))]["package_name"].tolist()
review_app['review_middle2_app'] = gp_data[(gp_data.review_total >= gp_data.review_total.quantile(0.6)) & (
        gp_data.review_total < gp_data.review_total.quantile(0.8))]["package_name"].tolist()
review_app['review_popular_app'] = gp_data[
    gp_data.review_total >=gp_data.review_total.quantile(0.8) ]["package_name"].tolist()

install_app={}

# 安装低于5000的app安装包名字
install_app['install_unpopular_app'] = gp_data[
    gp_data.install_total < 5000]["package_name"].tolist()
# 安装低于5000-30000的app安装包名字
install_app['install_unpopular_second_app'] = gp_data[(gp_data.install_total >= 5000) & (
        gp_data.install_total < 100000)]["package_name"].tolist()
# 安装低于10w-100w的app安装包名字
install_app['install_middle_app'] = gp_data[(gp_data.install_total >= 100000) & (
        gp_data.install_total < 1000000)]["package_name"].tolist()
# 安装过100万的app安装包名字
install_app['install_popular_app'] = gp_data[
    gp_data.install_total > 1000000]["package_name"].tolist()

popular_app_dict = {}

for app_category in gp_data["class_category"].unique():
    locals()[app_category+"_data"] = gp_data[gp_data.class_category ==app_category]

    locals()[app_category+"_install_unpopular_app"] = locals()[app_category+"_data"][(
                locals()[app_category+"_data"].review_total < locals()[app_category+"_data"].review_total.quantile(0.2))]["package_name"].tolist()
    locals()[app_category+"_install_unpopular_second_app"] = locals()[app_category+"_data"][
        (locals()[app_category+"_data"].review_total >=  locals()[app_category+"_data"].review_total.quantile(0.2))
        & (locals()[app_category+"_data"].review_total < locals()[app_category+"_data"].review_total.quantile(0.4))]["package_name"].tolist()
    locals()[app_category+"_install_middle_app"] = locals()[app_category+"_data"][
            (locals()[app_category+"_data"].review_total >= locals()[app_category+"_data"].review_total.quantile(0.4))
            & (locals()[app_category+"_data"].review_total< locals()[app_category+"_data"].review_total.quantile(0.6))]["package_name"].tolist()
    locals()[app_category+"_install_middle2_app"] = locals()[app_category+"_data"][
            (locals()[app_category+"_data"].review_total >= locals()[app_category+"_data"].review_total.quantile(0.6))
            & (locals()[app_category+"_data"].review_total< locals()[app_category+"_data"].review_total.quantile(0.8))]["package_name"].tolist()
    locals()[app_category+"_install_popular_app"] = locals()[app_category+"_data"][(
               locals()[app_category+"_data"].review_total >= locals()[app_category+"_data"].review_total.quantile(0.8))]["package_name"].tolist()

    popular_app_dict[app_category+"_install_unpopular_app"] = locals()[app_category+"_install_unpopular_app"]
    popular_app_dict[app_category+"_install_unpopular_second_app"] = locals()[app_category+"_install_unpopular_second_app"]
    popular_app_dict[app_category+"_install_middle_app"] = locals()[app_category+"_install_middle_app"]
    popular_app_dict[app_category+"_install_middle2_app"] = locals()[app_category+"_install_middle2_app"]
    popular_app_dict[app_category+"_install_popular_app"] = locals()[app_category+"_install_popular_app"] 
    gp_package_name = gp_data["package_name"].tolist()
    calss_category_dict = {
    package: category
    for package, category in zip(gp_package_name, gp_class_category)}

data_des=joblib.load('data_describe_fiance.pkl')

gp_data['describes_data']=gp_data['describes'].apply(lambda x :difflib.SequenceMatcher(None,data_des,x).quick_ratio())

In [32]:
def date_to_timestamp(date=None):
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    return timeStamp
def diff_days(date1=None, date2=None):
    return (date1 - date2) / 1000 / (3600 * 24)
def app_data_time(data=None):
    text = []
    cols = ['appName', 'firstInstallTime', 'lastUpdateTime', 'packageName']
    data["create_date_timemp"] = data["create_date"].apply(
                lambda x: date_to_timestamp(str(x)) * 1000)

    data['create_date']=data['create_date_timemp']
    data["install_to_create_days"] = data[[
        "firstInstallTime", "create_date"
    ]].apply(lambda x: diff_days(date1=x[1], date2=x[0]), axis=1)
    data["update_to_create_days"] = data[[
        "lastUpdateTime", "create_date"
    ]].apply(lambda x:diff_days(date1=x[1], date2=x[0]), axis=1)
    data["install_to_update_days"] = data[[
        "firstInstallTime", "lastUpdateTime"
    ]].apply(lambda x: diff_days(date1=x[1], date2=x[0]), axis=1)
    data["system_app"] = data[["appName", "packageName"]].apply(lambda x: 1 if x[0] == x[1] else 0, axis=1)
    mean_installdate, std_installdate = np.mean(
        data["install_to_create_days"]), np.std(data["install_to_create_days"])
    mean_updatedate, std_updatedate = np.mean(
        data["update_to_create_days"]), np.std(data["update_to_create_days"])
    # print("mean_installdate,std_installdate",mean_installdate,std_installdate)
    up_limit_install = data["install_to_create_days"].quantile(0.9)
    up_limit_update = data["update_to_create_days"].quantile(0.9)
    down_limit_install = data["install_to_create_days"].quantile(0.01)
    down_limit_update = data["update_to_create_days"].quantile(0.01)
    data["install_to_create_days"] = data["install_to_create_days"].apply(
        lambda x: up_limit_install
        if x > (mean_installdate + 3 * std_installdate) else x)
    data["update_to_create_days"] = data["update_to_create_days"].apply(
        lambda x: up_limit_update
        if x > (mean_updatedate + 3 * std_updatedate) else x)
    data["appName"] = data["appName"].apply(lambda x: str(x).lower())
    data["is_update"] = data["install_to_update_days"].apply(lambda x: 1 if x > 0 else 0)
    return data

In [33]:
describe_jieba=joblib.load('jiebaration.pkl')

In [34]:
llit=list(class_category.values())

In [35]:
describe_jieba

{'com.whatsapp': 0.18016928657799275,
 'com.instagram.android': 0.03993203058623619,
 'com.shopee.id': 0.8068965517241379,
 'com.facebook.katana': 0.13645876935956466,
 'com.facebook.orca': 0.1221001221001221,
 'com.google.android.apps.docs': 0.15366705471478465,
 'com.lazada.android': 0.6460193805814174,
 'com.google.android.videos': 0.042399639152007215,
 'com.google.android.apps.photos': 0.16611495636473067,
 'io.silvrr.installment': 0.7303930614504521,
 'cn.wps.moffice_eng': 0.07549962990377498,
 'com.bca': 0.700195100802081,
 'com.gojek.app': 0.8257231651924456,
 'com.ss.android.ugc.trill': 0.01966652415562206,
 'com.lenovo.anyshare.gps': 0.09380097879282219,
 'com.google.android.play.games': 0.03264964420259523,
 'com.google.android.apps.tachyon': 0.08174386920980926,
 'com.google.android.music': 0.6746099290780142,
 'com.telkomsel.telkomselcm': 0.6879188552798406,
 'com.grabtaxi.passenger': 0.125705683101242,
 'com.loan.cash.credit.easy.kilat.cepat.pinjam.uang.dana.rupiah': 0.70

In [36]:
app_level=joblib.load('app_level.pkl')

In [37]:
review_app1={}
install_app1={}
popular_app_dict1={}
app_level1={}

for i in review_app.keys():
    tt1=review_app[i]
    tt2=[i]*len(review_app[i])
    review_app1.update(zip(tt1,tt2))
for i in install_app.keys():
    tt1=install_app[i]
    tt2=[i]*len(install_app[i])
    install_app1.update(zip(tt1,tt2))
for i in popular_app_dict.keys():
    tt1=popular_app_dict[i]
    tt2=[i]*len(popular_app_dict[i])
    popular_app_dict1.update(zip(tt1,tt2))
for i in app_level.keys():
    tt1=app_level[i]
    tt2=[i]*len(app_level[i])
    app_level1.update(zip(tt1,tt2))

In [38]:
joblib.dump(review_app1,'review_app1.pkl')

['review_app1.pkl']

In [39]:
joblib.dump(install_app1,'install_app1.pkl')

['install_app1.pkl']

In [40]:
joblib.dump(popular_app_dict1,'popular_app_dict1.pkl')

['popular_app_dict1.pkl']

In [41]:
joblib.dump(app_level1,'app_level1.pkl')

['app_level1.pkl']

In [52]:
data00=pd.DataFrame(popular_app_dict1,index=[0]).T

In [88]:
set(review_app1.values())|set(install_app1.values())|set(popular_app_dict1.values())|set(app_level1.values())

{'Communications_install_middle2_app',
 'Communications_install_middle_app',
 'Communications_install_popular_app',
 'Communications_install_unpopular_app',
 'Communications_install_unpopular_second_app',
 'Education_install_middle2_app',
 'Education_install_middle_app',
 'Education_install_popular_app',
 'Education_install_unpopular_app',
 'Education_install_unpopular_second_app',
 'Entertainment_install_middle2_app',
 'Entertainment_install_middle_app',
 'Entertainment_install_popular_app',
 'Entertainment_install_unpopular_app',
 'Entertainment_install_unpopular_second_app',
 'Finance_install_middle2_app',
 'Finance_install_middle_app',
 'Finance_install_popular_app',
 'Finance_install_unpopular_app',
 'Finance_install_unpopular_second_app',
 'Game_install_middle2_app',
 'Game_install_middle_app',
 'Game_install_popular_app',
 'Game_install_unpopular_app',
 'Game_install_unpopular_second_app',
 'Lifestyle_install_middle2_app',
 'Lifestyle_install_middle_app',
 'Lifestyle_install_pop

In [56]:
def count_APP_num(data=None,llit=None):
    data_data_app=data[['category','loan_id']].groupby(by='category').count().T
    data_data_app['total_cnt']=data_data_app.T.sum()
    all_ot=set(llit)-set(data_data_app.columns)

    for i in all_ot:
        data_data_app[i]=0

    data_data_app_1=data_data_app.iloc[:,:-1]/data_data_app['total_cnt'].values[0]

    data_data_app_1.columns=data_data_app_1.columns+['ration_total']

    data_data_app=pd.concat([data_data_app,data_data_app_1],axis=1)
#     ind=0
#     zzu= list(itertools.permutations(llit,2))
#     for i in zzu:
#         data_data_app['_add_'.join(i)]=data_data_app[list(i)].apply(lambda x : x[list(i)[0]]+x[list(i)[1]],axis=1)

#         data_data_app['_minu_'.join(i)]=data_data_app[list(i)].apply(lambda x : x[list(i)[0]]-x[list(i)[1]],axis=1)

#         data_data_app['_division_'.join(i)]=data_data_app[list(i)].apply(lambda x : x[list(i)[0]]/x[list(i)[1]],axis=1)

#         data_data_app['_add_'.join(i)+'divisiontotal']=data_data_app['_add_'.join(i)].values[0]/data_data_app['total_cnt'].values[0]
#         data_data_app['_minu_'.join(i)+'divisiontotal']=data_data_app['_minu_'.join(i)].values[0]/data_data_app['total_cnt'].values[0] 
    return data_data_app

In [57]:
timess=['install_to_create_days','update_to_create_days']

In [71]:
from dateutil.parser import *
import pymongo
import time
import pandas as pd
import requests

In [60]:
start = time.time()
myclient = pymongo.MongoClient("mongodb:/*******************88888")
mycol = mydb["riskParamsNewDto"]
myDatetime=parse('2021-07-01 23:59:59')
myDatetime2=parse('2021-09-01 23:59:59')
x = mycol.find({"requestDate": {'$gt': myDatetime,'$lt': myDatetime2},"merchantCode":"x4_system","tLoanOrderDto.loanOrderStatus":5}).limit(1)
mu={}
ind=0
for i in x:
    ind=i['merchantTurnoverNo']
    mu[ind]=i
import joblib
end =time.time()
print("Running time: %s seconds"%(end - start))

Running time: 0.054963111877441406 seconds


In [72]:
def get_requests(url, encod=True):
    url = url
    headers = {
        "User-Agent":
            "Mozilla/5.0 (Windows NT 10.0; \
                       WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    if encod:
        response.encoding = 'utf-8'
    else:
        response.encoding = 'gbk'
    rs = response.text
    return rs

In [63]:
fulllink_data=pd.DataFrame(mu['rus1625161838403'][ 'tAppFullLinkInfoDto'],index=[mu['rus1625161838403'][ 'tAppFullLinkInfoDto']['loanId']])

In [67]:
 data_applist_json=pd.DataFrame(mu['rus1625161838403']['tAppUserApplistDto'],index=[mu['rus1625161838403']['tAppUserApplistDto']['loanId']])

In [73]:
app_data = pd.DataFrame(eval(get_requests(url=data_applist_json['applistJson'].iloc[0])))

In [76]:
app_data["create_date"] = mu['rus1625161838403']['requestDate']

app_data["loan_id"] =mu['rus1625161838403'][ 'tAppFullLinkInfoDto']['loanId']

In [77]:
data=app_data_time(data=app_data)

In [78]:
data["category"]=data['packageName'].map(calss_category_dict)

In [79]:
data["category"] = data["category"].fillna("others")

In [80]:
data_t=data

In [82]:
#a_app=joblib.load('/opt/jupyter_work/wuboyuan/X_系统更新/A_APP_DATA.pkl')
#a_app=a_app[a_app['loan_id']=='1022131dompet']

#data=app_data_time(data=a_app)
#data["category"]=data['packageName'].map(calss_category_dict)

#data["category"] = data["category"].fillna("others")

#data_t=data

In [83]:
col5=joblib.load('col5.pkl')

In [141]:
col5=list(set(review_app1.values())|set(install_app1.values())|set(popular_app_dict1.values())|set(app_level1.values())|set(['review_app_others','install_app_others','popular_app_others','app_leve_others']))

In [140]:
set(['review_app_others','install_app_others','popular_app_others','app_leve_others'])

{'app_leve_others',
 'install_app_others',
 'popular_app_others',
 'review_app_others'}

In [32]:
data_test=data_t[data_t[q]<=3000]

In [34]:
pd.DataFrame(data_test[q].describe()).T

,count,mean,std,min,25%,50%,75%,max
update_to_create_days,50.0,105.755421,88.407555,1.345582,12.134639,82.911164,198.54945,219.149824


In [ ]:
pd.DataFrame(data_test['install_to_create_days'].describe()).T

In [37]:
data_test[q]=data_test[q].astype(int)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
data_test[[q,'loan_id']].groupby(q).count().describe().T

,count,mean,std,min,25%,50%,75%,max
loan_id,31.0,1.612903,1.054433,1.0,1.0,1.0,2.0,4.0


In [ ]:
pd.DataFrame(data_test['install_to_create_days'].describe()).T

In [97]:
data['install_to_create_days']=data['install_to_create_days'].astype(int)

In [99]:
data['update_to_create_days']=data['update_to_create_days'].astype(int)

In [125]:
data_app_feat={}

In [126]:
data_app_feat['max_days']=data['install_to_create_days'].max()

data_app_feat['min_days']=data['install_to_create_days'].min()

data_app_feat['std_days']=data['install_to_create_days'].std()

data_app_feat['mean_days']=data['install_to_create_days'].mean()

data_app_feat['count']=data['install_to_create_days'].count()

In [127]:
data_nub=data[['loan_id','install_to_create_days']].groupby('install_to_create_days').count()

In [129]:
data_app_feat['max_nmbers']=data_nub['loan_id'].max()

data_app_feat['min_nmbers']=data_nub['loan_id'].min()

data_app_feat['std_nmbers']=data_nub['loan_id'].std()

data_app_feat['mean_nmbers']=data_nub['loan_id'].mean()

data_app_feat['count_nmbers']=data_nub['loan_id'].count()

In [153]:
data1_1=pd.DataFrame(data['packageName'].map(review_app1).fillna('review_app_others').value_counts()).T

data1_2=pd.DataFrame(data['packageName'].map(install_app1).fillna('install_app_others').value_counts()).T

data1_3=pd.DataFrame(data['packageName'].map(popular_app_dict1).fillna('popular_app_others').value_counts()).T

data1_4=pd.DataFrame(data['packageName'].map(app_level1).fillna('app_leve_others').value_counts()).T

data1_5=pd.concat([data1_1,data1_2,data1_3,data1_4],axis=1)

In [155]:
data1_5[mk]=0

In [157]:
data1_5[mk]

,others_install_middle_app,Finance_install_middle2_app,Education_install_unpopular_second_app,others_install_unpopular_second_app,Medical_install_unpopular_second_app,Office_install_unpopular_app,Lifestyle_install_unpopular_app,Lifestyle_install_unpopular_second_app,Medical_install_middle_app,others_install_middle2_app,...,review_unpopular_app,Game_install_middle_app,Finance_install_unpopular_app,Lifestyle_install_middle_app,Communications_install_middle_app,Communications_install_unpopular_second_app,Music_audio_install_middle_app,Medical_install_unpopular_app,Education_install_unpopular_app,Entertainment_install_unpopular_app
packageName,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [152]:
list(set(col5)-set(data1_5.columns))

[]

In [154]:
mk=list(set(col5)-set(data1_5.columns))

In [174]:
start = time.time()
ration_dict={}
indd=-10
#for j in [3000,30,7]:
j=7
data2=data[data['install_to_create_days']<j]
data1_1=pd.DataFrame(data2['packageName'].map(review_app1).fillna('review_app_others').value_counts()).T

data1_2=pd.DataFrame(data2['packageName'].map(install_app1).fillna('install_app_others').value_counts()).T

data1_3=pd.DataFrame(data2['packageName'].map(popular_app_dict1).fillna('popular_app_others').value_counts()).T

data1_4=pd.DataFrame(data2['packageName'].map(app_level1).fillna('app_leve_others').value_counts()).T

data1_5=pd.concat([data1_1,data1_2,data1_3,data1_4],axis=1)

mk=list(set(col5)-set(data1_5.columns))

data1_5[mk]=0

data_7days=data1_5




j=30000
data2=data[data['install_to_create_days']<j]
data1_1=pd.DataFrame(data2['packageName'].map(review_app1).fillna('review_app_others').value_counts()).T

data1_2=pd.DataFrame(data2['packageName'].map(install_app1).fillna('install_app_others').value_counts()).T

data1_3=pd.DataFrame(data2['packageName'].map(popular_app_dict1).fillna('popular_app_others').value_counts()).T

data1_4=pd.DataFrame(data2['packageName'].map(app_level1).fillna('app_leve_others').value_counts()).T

data1_5=pd.concat([data1_1,data1_2,data1_3,data1_4],axis=1)

mk=list(set(col5)-set(data1_5.columns))

data1_5[mk]=0 

data_7days
    #if indd<j:
        #print(5)
    #else:
        #for i in set(ration_dict.keys()):
            #data1_5/ration_dict[i]  
    #ration_dict[j]=data1_5
    #indd=j
end =time.time()
print("Running time: %s seconds"%(end - start))

Running time: 0.7138652801513672 seconds


In [175]:
start = time.time()
ration_dict={}
indd=-10
for j in [3000,7]:
    data2=data[data['install_to_create_days']<j]
    data1_1=pd.DataFrame(data2['packageName'].map(review_app1).fillna('review_app_others').value_counts()).T

    data1_2=pd.DataFrame(data2['packageName'].map(install_app1).fillna('install_app_others').value_counts()).T

    data1_3=pd.DataFrame(data2['packageName'].map(popular_app_dict1).fillna('popular_app_others').value_counts()).T

    data1_4=pd.DataFrame(data2['packageName'].map(app_level1).fillna('app_leve_others').value_counts()).T

    data1_5=pd.concat([data1_1,data1_2,data1_3,data1_4],axis=1)
    
    mk=list(set(col5)-set(data1_5.columns))
    
    data1_5[mk]=0
    
    #if indd<j:
        #print(5)
    #else:
        #for i in set(ration_dict.keys()):
            #data1_5/ration_dict[i]  
    #ration_dict[j]=data1_5
    #indd=j
end =time.time()
print("Running time: %s seconds"%(end - start))

Running time: 1.131941318511963 seconds


In [162]:
data1_5/ration_dict[21]

,Communications_install_middle2_app,Communications_install_middle_app,Communications_install_popular_app,Communications_install_unpopular_app,Communications_install_unpopular_second_app,Education_install_middle2_app,Education_install_middle_app,Education_install_popular_app,Education_install_unpopular_app,Education_install_unpopular_second_app,...,others_install_popular_app,others_install_unpopular_app,others_install_unpopular_second_app,popular_app_others,review_app_others,review_middle1_app,review_middle2_app,review_popular_app,review_unpopular_app,review_unpopular_second_app
packageName,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [ ]:
data1_5/

In [159]:
set(ration_dict.keys())

{7, 21, 60, 180}

In [ ]:
for i in ration_dict.keys():
    

In [130]:
data_app_feat

{'max_days': 399,
 'min_days': 8,
 'std_days': 157.8165279594477,
 'mean_days': 210.33333333333334,
 'count': 60,
 'max_nmbers': 17,
 'min_nmbers': 1,
 'std_nmbers': 2.5736125934912035,
 'mean_nmbers': 1.5384615384615385,
 'count_nmbers': 39}

In [134]:
set(review_app1.values())

{'review_middle1_app',
 'review_middle2_app',
 'review_popular_app',
 'review_unpopular_app',
 'review_unpopular_second_app'}

In [131]:
pd.DataFrame(data['packageName'].map(review_app1).fillna('review_app_others').value_counts()).T

,review_popular_app,review_app_others,review_middle2_app,review_middle1_app
packageName,45,9,5,1


In [106]:
data_app_feat

{'std_days': 157.8165279594477,
 'mean_days': 210.33333333333334,
 'max_days': 399,
 'min_days': 8}

In [133]:
def app_feats(data_t=None,llit=None):
    zz_data_0={}
    zz_data_0['days']={}
    zz_data_0['nmbers']={}
    zz_data_d={}
    zz_data_d1={}
    zz_data_d1_ration={}
    for q in timess:
        zz_data_0['days'][q]={}
        zz_data_0['nmbers'][q]={}
        data=data_t[data_t[q]<=3000]
        data[q]=data[q].astype(float)
        zz_data_0['days'][q]['all']=pd.DataFrame(data[q].describe()).T
        zz_data_0['days'][q]['all'].columns='all_'+q+'days_'+zz_data_0['days'][q]['all'].columns
        zz_data_0['nmbers'][q]['all']=data[[q,'loan_id']].groupby(q).count().describe().T
        zz_data_0['nmbers'][q]['all'].columns='all_'+q+'nmbers_'+zz_data_0['nmbers'][q]['all'].columns
        indd=-10
        mu=[indd]
        zz_data_d[q]={}
        zz_data_d1[q]={}
        zz_data_d1_ration[q]={}
        for j in [180,60,21,7]:
            data2=data[data[q]<j]
            zz_data_0['days'][q][j]=pd.DataFrame(data2[q].describe()).T
            zz_data_0['days'][q][j].columns=str(j)+q+'_days_'+zz_data_0['days'][q][j].columns
            zz_data_0['nmbers'][q][j]=data2[[q,'loan_id']].groupby(q).count().describe().T
            zz_data_0['nmbers'][q][j].columns=str(j)+q+'_nmbers_'+zz_data_0['nmbers'][q][j].columns
            zz_data=count_APP_num(data=data2,llit=llit)
            zz_data.columns=str(j)+q+zz_data.columns
            data1_1=pd.DataFrame(data2['packageName'].map(review_app1).fillna('review_app_others').value_counts()).T

            data1_2=pd.DataFrame(data2['packageName'].map(install_app1).fillna('install_app_others').value_counts()).T

            data1_3=pd.DataFrame(data2['packageName'].map(popular_app_dict1).fillna('popular_app_others').value_counts()).T

            data1_4=pd.DataFrame(data2['packageName'].map(app_level1).fillna('app_leve_others').value_counts()).T

            data1_5=pd.concat([data1_1,data1_2,data1_3,data1_4],axis=1)

            for i in set(col5)-set(data1_5.columns):
                data1_5[i]=0
            data1_5.columns=str(j)+q+data1_5.columns+'cnt'
            if indd<j:
                zz_data_d1[q][j]=data1_5 
            else:
                zz_data_d1[q][j]=data1_5 
                for m in mu[1:]:
                    cool=q+zz_data_d1[q][j].columns+'_'+zz_data_d1[q][m].columns+'ration'
                    ddl=zz_data_d1[q][m]
                    ddl.columns=zz_data_d1[q][j].columns
                    ddl2=zz_data_d1[q][j]/ddl
                    ddl2.columns=cool
                    zz_data_d1_ration[q][j]=ddl2



            zz_data[str(j)+q+'packageName_finance_cnt']=data2['packageName'].apply(lambda x: 1 if re.search(r'dana|cash|kredit|pinjaman|pinjam|dompet|ksp|ojk|darurat|tunai|modal|finance|cepat|keuangan|uang|loan|saku|tagihan|lender|money|uangku|kami|darurat|emas|loan|kantong|duit|rupiah|uangme|ekpressuang|pinjaman|kredi|uang|dan', x, re.I) else 0).sum()

            zz_data[str(j)+q+'packageName_gambling_cnt']=data2['packageName'].apply(lambda x: 1 if re.search(r'Mesinjudi|slot|Pertaruhan|perjudian|kasino', x, re.I) else 0).sum()

            zz_data[str(j)+q+'packageName_GPS_cnt']=data2['packageName'].apply(lambda x: 1 if re.search(r'clone|FakeGPS', x, re.I) else 0).sum()

            zz_data[str(j)+q+'alphabet_cnt']=data2['packageName'].apply(lambda x: 0 if re.search( '[A-Z]', x, re.I) else 1).sum()

            zz_data_d[q][j]=zz_data

            indd=j
            mu.append(j)
    t_data_1=pd.concat([zz_data_0['days']['install_to_create_days']['all'],zz_data_0['days']['install_to_create_days'][180],zz_data_0['days']['install_to_create_days'][60],zz_data_0['days']['install_to_create_days'][21],zz_data_0['days']['install_to_create_days'][7]],axis=1)

    t_data_2=pd.concat([zz_data_0['days']['update_to_create_days']['all'],zz_data_0['days']['update_to_create_days'][180],zz_data_0['days']['update_to_create_days'][60],zz_data_0['days']['update_to_create_days'][21],zz_data_0['days']['update_to_create_days'][7]],axis=1)

    t_data_3=pd.concat([zz_data_0['nmbers']['install_to_create_days']['all'],zz_data_0['nmbers']['install_to_create_days'][180],zz_data_0['nmbers']['install_to_create_days'][60],zz_data_0['nmbers']['install_to_create_days'][21],zz_data_0['nmbers']['install_to_create_days'][7]],axis=1)

    t_data_4=pd.concat([zz_data_0['nmbers']['update_to_create_days']['all'],zz_data_0['nmbers']['update_to_create_days'][180],zz_data_0['nmbers']['update_to_create_days'][60],zz_data_0['nmbers']['update_to_create_days'][21],zz_data_0['nmbers']['update_to_create_days'][7]],axis=1)

    t_data_5=pd.concat([zz_data_d['install_to_create_days'][180],zz_data_d['install_to_create_days'][60],zz_data_d['install_to_create_days'][21],zz_data_d['install_to_create_days'][7]],axis=1)

    t_data_6=pd.concat([zz_data_d['update_to_create_days'][180],zz_data_d['update_to_create_days'][60],zz_data_d['update_to_create_days'][21],zz_data_d['update_to_create_days'][7]],axis=1)

    t_data_7=pd.concat([zz_data_d1['install_to_create_days'][180],zz_data_d1['install_to_create_days'][60],zz_data_d1['install_to_create_days'][21],zz_data_d1['install_to_create_days'][7]],axis=1)

    t_data_8=pd.concat([zz_data_d1['update_to_create_days'][180],zz_data_d1['update_to_create_days'][60],zz_data_d1['update_to_create_days'][21],zz_data_d1['update_to_create_days'][7]],axis=1)

    t_data_9=pd.concat([zz_data_d1_ration['install_to_create_days'][60],zz_data_d1_ration['install_to_create_days'][21],zz_data_d1_ration['install_to_create_days'][7]],axis=1)

    t_data_10=pd.concat([zz_data_d1_ration['update_to_create_days'][60],zz_data_d1['update_to_create_days'][21],zz_data_d1_ration['update_to_create_days'][7]],axis=1)

    t_data_1.index=[data['loan_id'].iloc[0]]
    t_data_2.index=[data['loan_id'].iloc[0]]
    t_data_3.index=[data['loan_id'].iloc[0]]
    t_data_4.index=[data['loan_id'].iloc[0]]
    t_data_5.index=[data['loan_id'].iloc[0]]
    t_data_6.index=[data['loan_id'].iloc[0]]
    t_data_7.index=[data['loan_id'].iloc[0]]
    t_data_8.index=[data['loan_id'].iloc[0]]
    t_data_9.index=[data['loan_id'].iloc[0]]
    t_data_10.index=[data['loan_id'].iloc[0]]
    t_data=pd.concat([t_data_1,t_data_2,t_data_3,t_data_4,t_data_5,t_data_6,t_data_7,t_data_8,t_data_9,t_data_10],axis=1)
    return t_data